## Numerical Acceleration Techniques for Python based Quant Researchers

### Why Speed Matters in Quantitative Finance

The Black-Scholes formula shown below is a beautiful example of an **analytical solution** - a closed-form mathematical expression that gives us an exact answer instantly:

$$
\frac{\partial C}{\partial \tau} = \frac{1}{2}\sigma^2 S^2 \frac{\partial^2 C}{\partial S^2} + (r - q)S \frac{\partial C}{\partial S} - rC
$$

**But here's the reality:** Black-Scholes is the exception, not the rule.

In practice, most problems in quantitative finance have **no analytical solution**:
- American options (early exercise boundary is unknown)
- Path-dependent options (Asians, lookbacks, barriers with discrete monitoring)
- Multi-asset derivatives (basket options, spread options)
- Stochastic volatility models (Heston, SABR)
- Jump-diffusion processes (Merton, Kou)
- Credit derivatives with default correlation
- Real-world portfolio optimization with constraints

For these problems, we're forced to use **numerical methods**:
- Monte Carlo simulation (thousands to millions of paths)
- Finite difference schemes (large PDE grids)
- Binomial/trinomial trees (deep recursion)
- Numerical optimization (calibration, hedging)
- Machine learning models (training on massive datasets)

### The Need for Speed

As quantitative researchers, we live at the intersection of **mathematics, statistics, and practical implementation**. Our workflow is iterative:

1. **Prototype** a new model or pricing algorithm
2. **Refine** the numerical accuracy and stability
3. **Calibrate** to market data (often requiring thousands of evaluations)
4. **Backtest** strategies across years of historical data
5. **Deploy** for real-time risk management and trading

At every stage, **speed is everything**:
- **Research phase**: Faster code means more iterations, better models
- **Calibration**: 10x speedup turns a 10-minute fit into 1 minute
- **Production**: Real-time pricing of a large portfolio demands millisecond latency (pricing thousands of contracts simultaneously)
- **Risk management**: End-of-day VaR calculations across thousands of scenarios

Even a seemingly "fast" Python function called **millions of times** becomes a bottleneck. In production systems, we often need to price entire portfolios—**hundreds or thousands of contracts at once**—which demands vectorization and batch processing to avoid iterating through contracts one by one.

### What We'll Cover

In this notebook, we'll explore practical acceleration techniques that can give you **10x to 1000x speedups**:

1. **Vectorization** (NumPy) - leveraging optimized C/Fortran backends
2. **JIT compilation** (Numba) - turning Python loops into machine code
3. **GPU acceleration** (JAX, PyTorch) - massive parallelism for large-scale computations
4. **Parallel processing** (joblib) - distributing work across CPU cores
5. **Smart library choices** (comparing QuantLib, py_vollib, custom implementations)

We'll start simple with Black-Scholes as a benchmark, then build up to more realistic scenarios.

**Let's take a look.**

In [1]:
!pip install py_vollib
!pip install QuantLib

In [2]:
import datetime
import numpy as np
import QuantLib as ql
from scipy.stats import norm


def py_to_ql(dt):
    """Convert Python datetime.date to QuantLib Date"""
    if isinstance(dt, datetime.datetime):
        dt = dt.date()
    return ql.Date(dt.day, dt.month, dt.year)

### Generic timing wrapper function
We will use this to benchmark the two binomial models

In [3]:
import timeit
from time import time
from functools import wraps
import datetime as dt

def timing(f):
    @wraps(f)
    def wrap(*args, **kw):
        ts = time()
        result = f(*args, **kw)
        te = time()
        print('func:%r args:[%r, %r] took: %2.4f sec' % \
          (f.__name__, args, kw, te-ts))
        return result
    return wrap


def benchmark(func, runs=100, **kwargs):
    stmt = "func(**params)"
    globals_dict = {"func": func, "params": kwargs}

    # run the benchmark
    total = timeit.timeit(stmt, number=runs, globals=globals_dict)
    avg = total / runs

    # print results (similar style to your timing() decorator)
    print(
        f"func: {func.__name__} runs: {runs} time: total {total:.4f}s avg: {avg*1e6:.2f}µs"
    )

    return total, avg

In [4]:
# Wrapper for py_vollib
def py_vollib_price(S, K, T, r, sigma, flag='c'):
    import py_vollib.black_scholes as bs
    return bs.black_scholes(flag, S, K, T, r, sigma)

# Wrapper for QuantLib (European BS)
def quantlib_price(S, K, T, r, sigma, option_type="call"):
    import QuantLib as ql
    today = ql.Date.todaysDate()
    ql.Settings.instance().evaluationDate = today

    payoff = ql.PlainVanillaPayoff(
        ql.Option.Call if option_type=="call" else ql.Option.Put, K
    )
    maturity = today + int(T * 365)
    option = ql.VanillaOption(payoff, ql.EuropeanExercise(maturity))

    spot = ql.QuoteHandle(ql.SimpleQuote(S))
    flat_rf = ql.YieldTermStructureHandle(ql.FlatForward(today, r, ql.Actual365Fixed()))
    vol = ql.BlackVolTermStructureHandle(
        ql.BlackConstantVol(today, ql.NullCalendar(), sigma, ql.Actual365Fixed())
    )
    process = ql.BlackScholesProcess(spot, flat_rf, vol)
    option.setPricingEngine(ql.AnalyticEuropeanEngine(process))
    return option.NPV()

# Simple Black–Scholes Implementation
def black_scholes(S, K, r, sigma, T, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type.lower() == "call":
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

# Benchmark
total_mpy, avg_mpy = benchmark(black_scholes, runs=1_000_000,
                             S=100, K=105, T=0.5, r=0.03, sigma=0.25, option_type="call")
total_ql, avg_ql = benchmark(quantlib_price, runs=1_000_000,
                             S=100, K=105, T=0.5, r=0.03, sigma=0.25, option_type="call")
total_py, avg_py = benchmark(py_vollib_price, runs=1_000_000,
                             S=100, K=105, T=0.5, r=0.03, sigma=0.25, flag='c')

func: black_scholes runs: 1000000 time: total 34.2898s avg: 34.29µs
func: quantlib_price runs: 1000000 time: total 17.8551s avg: 17.86µs
func: quantlib_price runs: 1000000 time: total 17.8551s avg: 17.86µs
func: py_vollib_price runs: 1000000 time: total 1.0157s avg: 1.02µs
func: py_vollib_price runs: 1000000 time: total 1.0157s avg: 1.02µs


### Why is QuantLib so much slower?

QuantLib is built for **flexibility and production robustness**, not tight micro-benchmarks. It's like showing up in full mountaineering gear when everyone else is sprinting in running shoes.

**The overhead:**

1. **Heavy object construction every call** - evaluation date, payoff, exercise, option object, spot handle, rate curve, volatility surface, Black-Scholes process, and pricing engine
2. **Python ↔ C++ boundary crossings** - Every `Handle` and `shared_ptr` has overhead (smart pointer providing automatic memory management). Fine for one trade, terrible for 100k in a loop.
3. **Built for generality** - Supports arbitrary calendars, day count conventions, curve bootstrapping, stochastic processes, dozens of pricing engines. You pay for features you're not using.

**In contrast:**
- `py_vollib` is a thin wrapper over compiled C code
- Pure NumPy is just vectorized operations and two normal CDF calls
- No setup. No stateful objects. Just the math.

**The solution:** QuantLib wasn't meant to be called 100,000 times in a loop. Real workflows either vectorize or **reuse the process and engine** (see next cell).

In [5]:
import QuantLib as ql

def quantlib_engine(S, r, sigma):
    today = ql.Date.todaysDate()
    ql.Settings.instance().evaluationDate = today
    
    spot = ql.QuoteHandle(ql.SimpleQuote(S))
    flat_rf = ql.YieldTermStructureHandle(ql.FlatForward(today, r, ql.Actual365Fixed()))
    vol = ql.BlackVolTermStructureHandle(
        ql.BlackConstantVol(today, ql.NullCalendar(), sigma, ql.Actual365Fixed())
    )
    
    process = ql.BlackScholesProcess(spot, flat_rf, vol)
    engine = ql.AnalyticEuropeanEngine(process)
    return engine

def quantlib_option(S, K, T, option_type = "call"):
    today = ql.Date.todaysDate()
    payoff = ql.PlainVanillaPayoff(
        ql.Option.Call if option_type=="call" else ql.Option.Put, K
    )
    maturity = today + int(T * 365)
    option = ql.VanillaOption(payoff, ql.EuropeanExercise(maturity))
    return option

def quantlib_price_fast(engine, option):
    option.setPricingEngine(engine)
    return option.NPV()

In [6]:
engine = quantlib_engine(S=100, r=0.03, sigma=0.25)
option = quantlib_option(S=100, K=110, T=0.5)

total, avg = benchmark(quantlib_price_fast, runs=100000,
                       engine=engine, option=option)


func: quantlib_price_fast runs: 100000 time: total 0.1406s avg: 1.41µs


### The Real Problem

We don't need to just run 1 calculation. Often we want to solve a whole option pricing surface, inverting multiple prices to volatilities all at the same time. This is costly with the non-vectorised code as we will see. 

In [7]:
# Define price loop for simple python function
@timing
def price_loop_python(S, K, r, sigma, T):
    return [black_scholes(_S, K, r, sigma, T) for _S in S]

S_values = np.linspace(80, 120, 100_000)
prices = price_loop_python(S=S_values, K=105, r=0.03, sigma=0.25, T=0.5)

func:'price_loop_python' args:[(), {'S': array([ 80.        ,  80.0004    ,  80.00080001, ..., 119.99919999,
       119.9996    , 120.        ], shape=(100000,)), 'K': 105, 'r': 0.03, 'sigma': 0.25, 'T': 0.5}] took: 3.2420 sec


### Solution: Speedups and Vectorisation

In [8]:
!pip install jax
!pip install torch
!pip install numba

In [9]:
# Import core libraries
import os
os.environ['JAX_PLATFORMS'] = 'cpu'
import jax
import jax.numpy as jnp

import math
import torch
import numba as nb
import numpy as np
from scipy.stats import norm
from joblib import Parallel, delayed



### 1. Parallel CPU Execution

Parallelization (e.g., `joblib`, `multiprocessing`) uses multiple CPU
cores.

#### Key Benefits

-   Easy way to scale pure-Python code
-   Good for embarrassingly parallel workloads

#### Limitations

-   Has overhead from process spawning and data transfer
-   Not as fast as Numba for math-heavy loops

In [10]:
# Simple Black–Scholes Implementation
def black_scholes(S, K, r, sigma, T, option_type="call"):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)

    if option_type.lower() == "call":
        return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

    return K * np.exp(-r * T) * norm.cdf(-d2) - S * norm.cdf(-d1)

# Joblib Parallel version
def price_parallel_joblib(S, K, r, sigma, T, n_jobs=-1):
    return Parallel(n_jobs=n_jobs)(
        delayed(black_scholes)(_S, K, r, sigma, T)
        for _S in S
    )

### 2. Numba JIT (Just-In-Time Compilation)

Numba turns Python loops into optimized machine code using LLVM.

#### Key Benefits

-   Compiles math-heavy loops to native machine code
-   Removes Python interpreter overhead
-   Works extremely well for simulations, Monte Carlo, and tight loops

In [11]:
# Numba accelerated version
@nb.njit
def bs_numba(S, K, r, sigma, T):
    prices = np.empty_like(S)
    for i in range(len(S)):
        s = S[i]
        d1 = (math.log(s/K) + (r + 0.5*sigma*sigma)*T) / (sigma*math.sqrt(T))
        d2 = d1 - sigma*math.sqrt(T)
        N1 = 0.5 * (1 + math.erf(d1 / np.sqrt(2)))
        N2 = 0.5 * (1 + math.erf(d2 / np.sqrt(2)))
        prices[i] = s*N1 - K*math.exp(-r*T)*N2
    return prices

### 3. Vectorized NumPy

NumPy avoids Python loops by applying operations on entire arrays.

#### Key Benefits

-   Very high performance using C/Fortran backend
-   SIMD vectorization built-in
-   Clean and compact code

In [12]:
def norm_cdf_scipy(x):
    return norm.cdf(x)

def bs_price_vectorized(S, K, r, sigma, T, call=True):
    S = np.asarray(S)
    K = np.asarray(K)

    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)

    if call:
        return S*norm_cdf_scipy(d1) - K*np.exp(-r*T)*norm_cdf_scipy(d2)
    else:
        return K*np.exp(-r*T)*norm_cdf_scipy(-d2) - S*norm_cdf_scipy(-d1)

### 4. GPU Acceleration (CuPy / JAX / PyTorch)

GPUs run thousands of tiny parallel threads, ideal for large vectorized
workloads.

#### Key Benefits

-   Massive parallel compute (thousands of cores)
-   Extremely fast for Monte Carlo, PDE grids, and deep learning

#### Limitations

-   Requires vectorized array logic
-   Data transfer between CPU ↔ GPU can be costly
-   Not ideal for small workloads


In [13]:
# JAX Metal: needs python 3.10 at time of video
# pip install jax jaxlib jax-metal
# check gpu being identified. Otherwise uses CPU
jax.devices()

@jax.jit
def black_scholes_jax(S, K, r, sigma, T):
    d1 = (jnp.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * jnp.sqrt(T))
    d2 = d1 - sigma * jnp.sqrt(T)

    Nd1 = 0.5 * (1 + jax.scipy.special.erf(d1 / jnp.sqrt(2)))
    Nd2 = 0.5 * (1 + jax.scipy.special.erf(d2 / jnp.sqrt(2)))

    return S * Nd1 - K * jnp.exp(-r * T) * Nd2

# Torch GPU Implementation
device = "cuda" if torch.cuda.is_available() else "mps" # Metal Performance Shaders GPU

def black_scholes_torch(S, K, r, sigma, T):
    S = torch.as_tensor(S, device=device, dtype=torch.float32)

    d1 = (torch.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * torch.sqrt(torch.tensor(T)))
    d2 = d1 - sigma * torch.sqrt(torch.tensor(T))

    Nd1 = 0.5 * (1 + torch.erf(d1 / torch.sqrt(torch.tensor(2.0))))
    Nd2 = 0.5 * (1 + torch.erf(d2 / torch.sqrt(torch.tensor(2.0))))

    return S * Nd1 - K * torch.exp(torch.tensor(-r * T)) * Nd2

In [14]:
jax.devices()

[CpuDevice(id=0)]

### Run benchmark

In [15]:
# Define price loop for simple python function
def price_loop_python(S, K, r, sigma, T):
    return [black_scholes(_S, K, r, sigma, T) for _S in S]

N = 100_000 # number of contracts
S_values = np.linspace(80, 120, N)
S_jax = jnp.linspace(80, 120, N)
S_gpu = torch.linspace(80, 120, N, device=device)

# Option Terms
K = 100
r = 0.03
sigma = 0.25
T = 0.5

t_python = benchmark(price_loop_python, runs=10, S=S_values, K=K, r=r, sigma=sigma, T=T)
t_joblib = benchmark(price_parallel_joblib, runs=10, S=S_values, K=K, r=r, sigma=sigma, T=T)
t_numba  = benchmark(bs_numba, runs=10, S=S_values, K=K, r=r, sigma=sigma, T=T)
t_jax = benchmark(black_scholes_jax, runs=10, S=S_jax, K=K, r=r, sigma=sigma, T=T)
t_vectorized = benchmark(bs_price_vectorized, runs=10, S=S_values, K=K, r=r, sigma=sigma, T=T)
t_torch = benchmark(black_scholes_torch, runs=10, S=S_gpu, K=K, r=r, sigma=sigma, T=T)

print("---- Timing Results ----")
print(f"Joblib speedup     :  {t_python[0]/t_joblib[0]:.1f}x (depends on CPU cores!)")
print(f"Numba Speedup      :  {t_python[0]/t_numba[0]:.1f}x")
print(f"CPU JAX Speedup    :  {t_python[0]/t_jax[0]:.1f}x")
print(f"Vectorized speedup :  {t_python[0]/t_vectorized[0]:.1f}x")
print(f"GPU torch speedup  :  {t_python[0]/t_torch[0]:.1f}x")

func: price_loop_python runs: 10 time: total 32.2437s avg: 3224366.64µs
func: price_parallel_joblib runs: 10 time: total 6.0583s avg: 605832.33µs
func: bs_numba runs: 10 time: total 0.1197s avg: 11972.43µs
func: black_scholes_jax runs: 10 time: total 0.0418s avg: 4180.66µs
func: bs_price_vectorized runs: 10 time: total 0.0217s avg: 2169.87µs
func: black_scholes_torch runs: 10 time: total 0.0089s avg: 891.63µs
---- Timing Results ----
Joblib speedup     :  5.3x (depends on CPU cores!)
Numba Speedup      :  269.3x
CPU JAX Speedup    :  771.3x
Vectorized speedup :  1486.0x
GPU torch speedup  :  3616.3x
func: price_parallel_joblib runs: 10 time: total 6.0583s avg: 605832.33µs
func: bs_numba runs: 10 time: total 0.1197s avg: 11972.43µs
func: black_scholes_jax runs: 10 time: total 0.0418s avg: 4180.66µs
func: bs_price_vectorized runs: 10 time: total 0.0217s avg: 2169.87µs
func: black_scholes_torch runs: 10 time: total 0.0089s avg: 891.63µs
---- Timing Results ----
Joblib speedup     :  5.3x 

In [17]:
N = 20_000_000 # number of contracts
S_values = np.linspace(80, 120, N)
S_jax = jnp.linspace(80, 120, N)
S_gpu = torch.linspace(80, 120, N, device=device)

t_numba  = benchmark(bs_numba, runs=25, S=S_values, K=K, r=r, sigma=sigma, T=T)
t_vectorized = benchmark(bs_price_vectorized, runs=25, S=S_values, K=K, r=r, sigma=sigma, T=T)
t_jax = benchmark(black_scholes_jax, runs=25, S=S_jax, K=K, r=r, sigma=sigma, T=T)
t_torch = benchmark(black_scholes_torch, runs=25, S=S_gpu, K=K, r=r, sigma=sigma, T=T)

print("---- Timing Results ----")
print(f"Vectorized speedup :  {t_numba[0]/t_vectorized[0]:.1f}x")
print(f"CPU JAX Speedup    :  {t_numba[0]/t_jax[0]:.1f}x")
print(f"GPU torch speedup  :  {t_numba[0]/t_torch[0]:.1f}x")

func: bs_numba runs: 25 time: total 17.4695s avg: 698778.92µs
func: bs_price_vectorized runs: 25 time: total 12.7947s avg: 511788.15µs
func: black_scholes_jax runs: 25 time: total 0.0288s avg: 1151.45µs
func: black_scholes_torch runs: 25 time: total 0.0015s avg: 58.41µs
---- Timing Results ----
Vectorized speedup :  1.4x
CPU JAX Speedup    :  606.9x
GPU torch speedup  :  11964.0x
func: bs_price_vectorized runs: 25 time: total 12.7947s avg: 511788.15µs
func: black_scholes_jax runs: 25 time: total 0.0288s avg: 1151.45µs
func: black_scholes_torch runs: 25 time: total 0.0015s avg: 58.41µs
---- Timing Results ----
Vectorized speedup :  1.4x
CPU JAX Speedup    :  606.9x
GPU torch speedup  :  11964.0x
